This is an interactive plot of stresses around a wellbore calculated with the Kirsch solution.
The script also checks for tensile and shear failure.
Written by dnicolasespinoza @ github

In [17]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

from ipywidgets import interact

In [18]:
@interact(SHmax=(7000,10000,20),Shmin=(6000,7000,20),Pw=(4000,8000,20)) # Widget variables 
def plotter(SHmax=8000,Shmin=6000,Pw=4000):
    # define linear space for theta
    theta = np.linspace(1,360,90) # [degrees] angle from SHmax
    a = 0.5 # wellbore radius [ft]
    r = 0.5 # distance from wellbore
    # define a few more variables
    Ts = 200 # [psi] rock tensile strength
    UCS = 9000 # [psi] rock UCS
    q = 3 # friction parameter (1+sin(phi))/(1-sin(phi))
    Pp = 4000 # [psi] pore pressure
    sigHmax = SHmax - Pp
    sighmin = Shmin - Pp
    # calculate hoop and radial stress  
    sigma_rr = (Pw-Pp)*(a**2/r**2) + (sigHmax+sighmin)/2*(1-a**2/r**2) + (sigHmax-sighmin)/2*(1-4*a**2/r**2+3*a**4/r**4)*np.cos(2*theta*3.1415/180)    
    sigma_tt = -(Pw-Pp)*(a**2/r**2) + (sigHmax+sighmin)/2*(1+a**2/r**2) - (sigHmax-sighmin)/2*(1+3*a**4/r**4)*np.cos(2*theta*3.1415/180)
    #check failure
    #Ts
    index = 0
    TensFrac = np.zeros(len(theta))
    for x in sigma_tt:
        if x < (-Ts):
            TensFrac[index] = 1
        index = index + 1
    #shear
    ShearFrac = np.zeros(len(theta))
    sigma_1avail = UCS + q*sigma_rr
    index2 = 0
    for x in sigma_tt:
        if x > sigma_1avail[index2]:
            ShearFrac[index2] = 1
        index2 = index2 + 1
    ### plotting
    plt.subplot(211) # plot pore pressure, vertical stress and seafloor
    plt.plot(theta,sigma_rr, 'b-', label = "sigma_rr")
    plt.plot(theta,sigma_tt, 'r-', label = "sigma_tt")
    # plotting options
    plt.xlabel('theta [deegres]')
    plt.ylabel('Stress [psi] (Pp=4000psi)')
    plt.xlim(0,360)
    plt.ylim(-5000,15000)
    #plt.gca().invert_yaxis()
    plt.legend()
    plt.subplot(212) # check failure criterion
    plt.plot(theta,TensFrac, 'b+', label = "Tensile Fractures")
    plt.plot(theta,ShearFrac, 'rx', label = "Shear Failure")
    # plotting options
    plt.xlabel('theta [deegres]')
    plt.ylabel('Failure [1:YES, 0;NO]')
    plt.xlim(0,360)
    plt.legend()
    ### debugging
    #print(theta)
    #print(sigma_rr)
    #print(TensFrac)

interactive(children=(IntSlider(value=8000, description='SHmax', max=10000, min=7000, step=20), IntSlider(valu…